Before running the code blocks, run the following commands to get the kernel prepared:
```sh
julia
import Pkg
Pkg.update()
Pkg.precompile()

using Pkg
Pkg.build("IJulia")
```

In [200]:
using HDF5
using Serialization
using DataFrames, Plots, Statistics
using Plots.PlotMeasures 
using DataFramesMeta
using Missings

In [201]:
has_data(v) = any(!ismissing, v)
safe_mean(v) = has_data(v) ? mean(skipmissing(v)) : missing
# safe_std(v) = has_data(v) ? max(0, std(collect(skipmissing(v)); corrected=false)) : missing
function safe_std(v)
    if has_data(v)
        result = std(collect(skipmissing(v)); corrected=false)
        if result < 0
            println("Warning: negative std encountered, setting to 0.0")
            result = 0.0
        end
        return abs(result)
    else
        return missing
    end
end
safe_min(v)  = has_data(v) ? minimum(skipmissing(v)) : missing
safe_max(v)  = has_data(v) ? maximum(skipmissing(v)) : missing


safe_max (generic function with 1 method)

In [202]:
function rename_regime(trial)
    if trial == "1(1.0) particles"
        return "Extreme degeneracy"
    elseif trial == "1(0.999) particles"
        return "High degeneracy"
    elseif trial == "1(0.99) particles"
        return "Near-degeneracy(0.99)"
    elseif trial == "50%(1.0) particles"
        return "Balanced regime"
    elseif trial == "all(1.0) particles"
        return "Uniform regime"
    end
    return trial
end

rename_regime (generic function with 1 method)

In [203]:

function timer_dict_to_df(timer_dict, optimization, flatten=false)
    rows = []

    for (trial, ranks) in timer_dict
        k, total_rank, nprt_per_rank, n_float_per_particle, perm, p = split(trial, ":")
        k = k == "half" ? "50%" : k
        trial_name = "$k($p) particles"
        for (rank, timers) in ranks
            function format_row(op, metric, value)
                return (
                    optimization = optimization,
                    trial  = String(trial_name),
                    perm = String(perm),
                    total_rank = parse(Int, total_rank),
                    particle_size = n_float_per_particle,
                    nprt_per_rank = nprt_per_rank,
                    rank   = string(rank),
                    op     = String(op),
                    metric = String(metric),
                    value  = value,
                )
            end
            function recursive_push(inner_timers)
                for (op, metrics) in inner_timers
                    if op == "receive loop" && flatten
                        for (inner_op, inner_metrics) in metrics["inner_timers"]
                            for (metric, value) in inner_metrics
                                push!(rows, format_row(inner_op, metric, value))
                            end
                        end
                    else
                        push!(rows, format_row(op, "time_s", metrics["time_ns"] / 1e9))
                        push!(rows, format_row(op, "n_calls", metrics["n_calls"]))
                        recursive_push(metrics["inner_timers"])
                    end
                end
            end
            push!(rows, format_row("overall", "time_s", timers["time_ns"] / 1e9))
            push!(rows, format_row("overall", "n_calls", timers["n_calls"]))
            recursive_push(timers["inner_timers"])
        end
    end
    return DataFrame(rows)
end

timer_dict_to_df (generic function with 3 methods)

In [204]:
# function render(df, label=false)
#   trials = unique(df.trial)
#   ops    = unique(df.op)

#   ntrials, nops = length(trials), length(ops)

#   # 2) Create an empty grid, with row & column titles
#   plt = plot(
#     layout     = (ntrials, nops),
#     legend = label ? :topleft : :none,
#     size       = (300 * nops, 300 * ntrials),
#   )

#   # 3) Fill it in
#   for (i, trial) in enumerate(trials)
#     for (j, op) in enumerate(ops)
#       sub = df[(df.trial .== trial) .& (df.op .== op), :]
#       for rk in unique(sub.rank)
#         subrk = sub[sub.rank .== rk, :]
#         scatter!(
#           plt[i, j],
#           string.(subrk.total_rank),    # categorical x
#           subrk.time_ms;
#           marker     = :circle,
#           markersize = 4,
#           label      = rk,
#           right_margin = 10mm,
#           color      = rk == "0" ? :blue : :orange,
#         )
#       end
#       plot!(plt[i, j], title = string(op), top_margin = 10mm)
#       if j == 1
#         plot!(plt[i, j], ylabel = string(trial), left_margin = 20mm)
#       end

#       # only the bottom row gets the shared x‐axis label
#       if i == ntrials
#         plot!(plt[i, j], xlabel = "Total Rank")
#       end
#     end
#   end

#   display(plt)
#   return plt
# end

In [205]:
function df_from_h5(root_dir, category)
    optimization = split(category, "/")[end]

    root = dirname(@__FILE__)
    h5path(rank) = joinpath(root, "../$(root_dir)/$(category)/", "all_timers_$(rank).h5") 
    all_timer_dfs = DataFrame()
    for rank in [2, 4, 8, 16, 32]
        blob = h5open(h5path(rank)) do f
            read(f, "all_timers")
        end

        # Deserialize back into Dict{String,Dict{Int,Dict{String,Any}}}
        merged_timers = deserialize(IOBuffer(blob))

        # Convert to DataFrame
        timer_df = timer_dict_to_df(merged_timers, optimization)
        
        # Concat to all_timer_dfs
        all_timer_dfs = vcat(all_timer_dfs, timer_df)
    end

    wide = unstack(
    all_timer_dfs,
    [:optimization, :trial, :perm, :total_rank, :nprt_per_rank, :particle_size, :rank, :op],
    :metric,
    :value
    )
    # Sort by perm and trial
    wide = sort!(wide, [:perm, :trial])

    first_df = wide[wide.perm .== "firstperm", :]
    rand_df = wide[wide.perm .== "randperm", :]
    return (first_df, rand_df)
end

df_from_h5 (generic function with 1 method)

In [206]:

function widen_ops(df::DataFrame)
    rename!(df, Symbol.(names(df)))

    keys   = [:optimization, :trial, :perm, :total_rank, :nprt_per_rank, :particle_size, :rank]
    wanted = ["overall","waitall","broadcast","copy states","write from buffer","write to buffer", "waitall phase", "buffer write-back",
              "receive loop","send loop","irecv","remote duplicates copy", "optimize resample", "local copy", "remote receive", "send plan", "receive plan", "local replication", "local replication", "remote replication"]

    df2 = subset(df, :op => ByRow(in(wanted)))

    transform!(df2, :n_calls => ByRow(x -> x == 0 ? missing : x) => :n_calls)

    g = groupby(df2, vcat(keys, [:op]))
    avg = combine(g,
        [:time_s, :n_calls] => ((t, c) -> sum(t) / sum(c) ) => :time_s_per_call
    )

    wide = unstack(avg, keys, :op, :time_s_per_call; combine=first)
    rename!(wide, Symbol.(replace.(string.(names(wide)), r"[ -]" => "_")))
    return wide
end

widen_ops (generic function with 1 method)

In [207]:
function stats(df_stats)
    df_stats = @chain df_stats[df_stats.total_rank .> 1, :] begin
        @transform(
            :waitall_ratio = :waitall_phase ./ :overall,
            :localcopy_ratio = :local_replication ./ :overall,
            :remotedup_ratio = :remote_replication ./ :overall,
            :writefrombuf_ratio = :buffer_write_back ./ :overall,
            :resample_ratio = :optimize_resample ./ :overall,
            # :overall_time_log = log2.(:overall),
            # :waitall_time_log = log2.(:waitall_phase),
            # :copy_states_time_log = log2.(:copy_states),
            # :resample_time_log = log2.(:optimize_resample)
        )
    end
    return df_stats
end

stats (generic function with 1 method)

In [208]:
function rename_stats(metric)
    if metric == "waitall_ratio"
        return "Waitall/Overall Time Ratio"
    elseif metric == "overall"
        return "Overall Time (s)"
    elseif metric == "copy_states"
        return "Copy States Time (s)"
    elseif metric == "waitall_phase"
        return "Waitall Time (s)"
    elseif metric == "optimize_resample"
        return "Optimise Resample (s)"
    elseif metric == "resample_ratio"
        return "Overhead Time Ratio"
    else
        return metric
    end
end

function rename_optimization(optimization)
    if optimization == "original"
        return "Baseline"
    elseif optimization == "only_optimize_resampling"
        return "Optimise Resampling Only"
    elseif optimization == "only_dedup"
        return "Deduplication Only"
    elseif optimization == "dedup_threading"
        return "Deduplication + Threading"
    elseif optimization == "dedup_threading_optimize_resampling"
        return "Full Optimisation"
    else
        return optimization
    end
end

rename_optimization (generic function with 1 method)

In [209]:
function render_stats(df, stats_to_plot=["overall", "copy_states", "waitall_phase", "waitall_ratio"])
    # Configuration for plots
    trials = ["1(1.0) particles", "1(0.99) particles", "50%(1.0) particles", "all(1.0) particles"]
    optimizations = unique(df.optimization)
    ntrials = length(trials)
    nstats = length(stats_to_plot)
    n_opts = length(optimizations)

    # --- Create the individual plots for the grid ---
    axes = []
    for (i, trial) in enumerate(trials)
        for (j, stat) in enumerate(stats_to_plot)
            # Create a new plot object for this subplot
            p = plot(legend=false, palette=:auto, bottom_margin=10mm)
            
            # --- Set conditional labels and titles ---
            if i == 1
                plot!(p, title = rename_stats(stat), top_margin = 10mm)
            end
            if j == 1
                plot!(p, ylabel = rename_regime(trial), left_margin = 20mm)
            end
            if i == ntrials
                plot!(p, xlabel = "Total Rank", bottom_margin = 10mm)
            end

            df_filtered = df[df.trial .== trial, :]
            
            # --- Add each optimization as a series to the plot ---
            for optimization in optimizations
                sub = select(df_filtered, :optimization, :total_rank, :rank, stat => :value)
                subrk = sub[sub.optimization .== optimization, :]
                isempty(subrk) && continue

                # Determine if log scale should be used
                use_log = stat in ["overall", "copy_states", "waitall_phase"]

                # Group and calculate statistics.
                g = if use_log
                    # Add a small epsilon to avoid log(0) issues
                    subrk_log = @transform(subrk, :log_value = log10.(:value .+ 1e-12))
                    
                    # Calculate stats in both linear (for mean) and log (for std) space
                    g_linear = combine(groupby(subrk, :total_rank), :value => safe_mean => :mean)
                    g_log = combine(groupby(subrk_log, :total_rank), :log_value => safe_std  => :std_log)
                    
                    # Join them together and return
                    leftjoin(g_linear, g_log, on = :total_rank)
                else
                    combine(groupby(subrk, :total_rank),
                            :value => safe_mean => :mean,
                            :value => safe_std  => :std)
                end

                # Filter out rows with missing data and sort
                filter!(row -> all(!ismissing, values(row)), g)
                sort!(g, :total_rank)
                isempty(g) && continue
                
                # Prepare plot variables (mean and ribbon) based on the scale
                local plot_mean, ribbon_val
                if use_log
                    plot!(p, yaxis=:log)
                    # Use the arithmetic mean for the central line
                    plot_mean = g.mean
                    
                    # Calculate the geometric standard deviation as a multiplicative factor
                    gstd = 10 .^ g.std_log
                    
                    # Define ribbon bounds multiplicatively around the arithmetic mean
                    lower_bound = plot_mean ./ gstd
                    upper_bound = plot_mean .* gstd
                    
                    # The ribbon is the distance from the central line
                    ribbon_val = (plot_mean .- lower_bound, upper_bound .- plot_mean)
                else
                    plot_mean = g.mean
                    ribbon_val = g.std
                end

                plot!(p, string.(g.total_rank), plot_mean,
                      ribbon = ribbon_val, seriestype = :path,
                      markersize = 4, linewidth = 1.5)
            end
            push!(axes, p)
        end
    end

    # --- Create the global legend ---
    legend_labels = permutedims([rename_optimization(opt) for opt in optimizations])
    legend_plot = plot(
        (1:n_opts)', # Dummy data for legend entries
        legend = :top,
        legend_columns = -1, # Force a single horizontal row
        labels = legend_labels,
        framestyle = :none,
        palette = :auto,
        size = (1,1)
    )

    # --- Combine legend and plots into the final figure ---
    final_fig = plot(
        legend_plot,
        axes...,
        layout = @layout([A{0.05h}; grid(ntrials, nstats)]),
        size = (400 * nstats, 300 * ntrials)
    )

    return final_fig
end

render_stats (generic function with 3 methods)

In [210]:
function render_overhead(df, stats_to_plot=["optimize_resample", "resample_ratio"])
    # Configuration for plots
    trials = ["1(1.0) particles", "1(0.99) particles", "50%(1.0) particles", "all(1.0) particles"]
    nstats = length(stats_to_plot)
    ntrials = length(trials)

    # --- Create the individual plots for the row ---
    axes = []
    # Filter the DataFrame for the single optimization we care about
    df_opt_filtered = df
    
    for (j, stat) in enumerate(stats_to_plot)
        # Create a new plot object for this subplot
        p = plot(legend=false, palette=:auto, bottom_margin=10mm, title=rename_stats(stat), top_margin=10mm)
        
        # Add common labels
        plot!(p, xlabel = "Total Rank")
        # if j == 1
        #     plot!(p, ylabel = "Value", left_margin=20mm)
        # end

        # --- Add each trial as a series to the plot ---
        for trial in trials
            sub = select(df_opt_filtered[df_opt_filtered.trial .== trial, :], :total_rank, :rank, stat => :value)
            isempty(sub) && continue

            # Determine if log scale should be used based on the stat name
            use_log = stat in []

            # Group and calculate statistics
            g = if use_log
                sub_log = @transform(sub, :log_value = log10.(:value .+ 1e-12))
                g_linear = combine(groupby(sub, :total_rank), :value => safe_mean => :mean)
                g_log = combine(groupby(sub_log, :total_rank), :log_value => safe_std  => :std_log)
                leftjoin(g_linear, g_log, on = :total_rank)
            else
                combine(groupby(sub, :total_rank),
                        :value => safe_mean => :mean,
                        :value => safe_std  => :std)
            end

            filter!(row -> all(!ismissing, values(row)), g)
            sort!(g, :total_rank)
            isempty(g) && continue
            
            # Prepare plot variables
            local plot_mean, ribbon_val
            if use_log
                plot!(p, yaxis=:log)
                plot_mean = g.mean
                gstd = 10 .^ g.std_log
                lower_bound = plot_mean ./ gstd
                upper_bound = plot_mean .* gstd
                ribbon_val = (plot_mean .- lower_bound, upper_bound .- plot_mean)
            else
                plot_mean = g.mean
                ribbon_val = g.std
            end

            plot!(p, string.(g.total_rank), plot_mean,
                  ribbon = ribbon_val, seriestype = :path,
                  markersize = 4, linewidth = 1.5)
        end
        push!(axes, p)
    end

    # --- Create the global legend for trials ---
    legend_labels = permutedims([rename_regime(t) for t in trials])
    legend_plot = plot(
        (1:ntrials)',
        legend = :top,
        legend_columns = -1,
        labels = legend_labels,
        framestyle = :none,
        palette = :auto,
        size = (1,1)
    )

    # --- Combine legend and plots ---
    final_fig = plot(
        legend_plot,
        axes...,
        layout = @layout([A{0.1h}; grid(1, nstats)]),
        size = (450 * nstats, 400)
    )

    return final_fig
end


render_overhead (generic function with 3 methods)

In [211]:
import Pkg; Pkg.add("ColorSchemes")
using ColorSchemes

   Resolving package versions...
  No Changes to `~/Desktop/UCL/masterproj/ParticleDA.jl/extra/Project.toml`
  No Changes to `~/Desktop/UCL/masterproj/ParticleDA.jl/extra/Manifest.toml`


In [212]:
# root_dir = "test/output29"
# _, original_100k_df = df_from_h5(root_dir, "original")
# function stats(df_stats)
#     df_copy_states = df_stats[df_stats.op .== "copy states", :]
#     rename!(df_copy_states, :time_s => :copy_states_time_s)
#     df_stats = df_stats[(df_stats.op .!= "overall") .& (df_stats.op .!= "broadcast"), :]
#     df_stats = leftjoin(df_stats, df_copy_states, on=[:optimization, :trial, :total_rank, :rank], makeunique=true)
#     df_stats = @chain df_stats[df_stats.total_rank .> 1, :] begin
#         @transform(
#             :time_ratio = :time_s ./ :copy_states_time_s
#         )
#     end
#     return df_stats
# end
# original_100k_df = stats(original_100k_df)

# function render_stats_refactored(df)
#     # Configuration for plots
#     trials = ["1(1.0) particles", "1(0.99) particles",
#               "50%(1.0) particles", "all(1.0) particles"]
#     stats_metrics = ["copy states", "waitall phase", "buffer write-back",
#                      "receive loop","send loop","send plan", "receive plan", "local replication", "remote replication"]
#     # filter out metrics not present in the DataFrame
#     stats_metrics = filter(metric -> any(df.op .== metric), stats_metrics)
#     nstats = length(stats_metrics)

#     # Helper to rename metrics for the legend
#     function rename_stats(op)
#         if op == "waitall"
#             return "waitall phase"
#         elseif op == "write from buffer"
#             return "buffer write-back"
#         end
#         return op
#     end
    
#     # Get a consistent color palette for all series
#     colors = get_color_palette(ColorSchemes.Set1_9, plot_color(:white))[1:nstats]
    
#     # 1. Create each of the four subplots individually
#     axes = []
#     for (i, trial) in enumerate(trials)
#         # Create a blank plot for the current trial
#         p = plot(
#             title = rename_regime(trial), 
#             xlabel = "Total Rank",
#             bottom_margin = 10mm,
#         )
        
#         # Add Y-axis label only to the plots in the first column
#         if i in [1, 3]
#             plot!(p, ylabel = "Time Breakdown", left_margin = 15mm)
#         end

#         df_filtered = df[df.trial .== trial, :]

#         # Add each metric as a series to the plot
#         for (j, metric) in enumerate(stats_metrics)
#             subrk = df_filtered[df_filtered.op .== metric, :]
            
#             # Skip if there's no data for this metric
#             isempty(subrk) && continue

#             g = combine(groupby(subrk, :total_rank),
#                         :time_ratio => safe_mean => :mean,
#                         :time_ratio => safe_std  => :std)
#             filter!(row -> !ismissing(row.mean) && !ismissing(row.std), g)
#             sort!(g, :total_rank)

#             isempty(g) && continue

#             plot!(
#                 p,
#                 string.(g.total_rank),
#                 g.mean,
#                 ribbon = g.std,
#                 seriestype = :path,
#                 markersize = 4,
#                 linewidth = 1.5,
#                 label = "", # Disable individual labels
#                 color = colors[j]
#             )
#         end
#         push!(axes, p)
#     end

#     # 2. Create a separate plot object that only contains the legend
#     legend_labels = permutedims([rename_stats(s) for s in stats_metrics])
#     legend_plot = plot(
#         (1:nstats)', 
#         legend_position = :outertop, 
#         legend_columns = 3, 
#         labels = legend_labels, 
#         colors = colors, 
#         framestyle = :none,
#         palette = colors,
#     )

#     # 3. Combine the legend and the subplots into a final figure
#     # The layout places the legend ('l') in a small top row,
#     # followed by a 2x2 grid of the other plots.
#     final_fig = plot(
#         legend_plot,
#         axes...,
#         layout = @layout([l{0.1h}; grid(2, 2)]),
#         size = (900, 750)
#     )

#     return final_fig
# end
# render_stats_refactored(original_100k_df)
# savefig("copy_states_original_time_breakdown.pdf")

In [ ]:
root_dir = "test/output29"
_, original_100k_df = df_from_h5(root_dir, "original")
_, dedup_100k_df = df_from_h5(root_dir, "only_dedup")
_, dedup_threads_100k_df = df_from_h5(root_dir, "dedup_threading")
# _, op_100k_df = df_from_h5(root_dir, "only_optimize_resampling")
_, dtop_100k_df = df_from_h5(root_dir, "dedup_threading_optimize_resampling")

union_df = vcat(original_100k_df, dedup_100k_df, dedup_threads_100k_df, dtop_100k_df)
union_df = widen_ops(union_df)
union_df = stats(union_df)
println("Data loaded and processed.")

# stats
# stats_df = combine(groupby(union_df, [:optimization, :trial, :total_rank]), :overall => mean => :overall_mean)
# original_stats = stats_df[stats_df.optimization .== "original", [:trial, :total_rank, :overall_mean]]
# rename!(original_stats, :overall_mean => :original_overall_mean)
# stats_df = leftjoin(stats_df, original_stats, on=[:trial, :total_rank])
# stats_df = @chain stats_df begin
#     @transform(
#         improvement = (:original_overall_mean .- :overall_mean) ./ :original_overall_mean * 100
#     )
#     sort!([:trial, :total_rank, :optimization])
# end
# stats_df = stats_df[stats_df.optimization .== "dedup_threading_optimize_resampling", :]
# render_stats(union_df)
# render_overhead(union_df, ["optimize_resample", "resample_ratio"])

# println(union_df[union_df.total_rank .== 32 .&& union_df.optimization .== "original", :])
savefig("overhead.pdf")

"/Users/sunchenge/Desktop/UCL/masterproj/ParticleDA.jl/extra/overhead.pdf"

In [214]:
# function timer_dict_to_df(timer_dict, optimization, total_rank, flatten=false)
#     rows = []

#         for (rank, timers) in timer_dict
#             function format_row(op, metric, value)
#                 return (
#                     optimization = optimization,
#                     total_rank = total_rank,
#                     rank   = string(rank),
#                     op     = String(op),
#                     metric = String(metric),
#                     value  = value,
#                 )
#             end
#             function recursive_push(inner_timers)
#                 for (op, metrics) in inner_timers
#                     if op == "receive loop" && flatten
#                         for (inner_op, inner_metrics) in metrics["inner_timers"]
#                             for (metric, value) in inner_metrics
#                                 push!(rows, format_row(inner_op, metric, value))
#                             end
#                         end
#                     else
#                         push!(rows, format_row(op, "time_s", metrics["time_ns"] / 1e9))
#                         push!(rows, format_row(op, "n_calls", metrics["n_calls"]))
#                         recursive_push(metrics["inner_timers"])
#                     end
#                 end
#             end
#             push!(rows, format_row("End-to-End", "time_s", timers["total_time_ns"] / 1e9))
#             push!(rows, format_row("End-to-End", "n_calls", 1))
#             recursive_push(timers["inner_timers"])
#         end
#     return DataFrame(rows)
# end

# function df_from_h5(root_dir, category)
#     optimization = split(category, "/")[end]

#     root = dirname(@__FILE__)
#     h5path(rank) = joinpath(root, "../$(root_dir)/$(category)/", "all_timers_$(rank).h5") 
#     all_timer_dfs = DataFrame()
#     for rank in [2, 4, 8, 16]
#         blob = h5open(h5path(rank)) do f
#             read(f, "all_timers")
#         end

#         # Deserialize back into Dict{String,Dict{Int,Dict{String,Any}}}
#         merged_timers = deserialize(IOBuffer(blob))
#         # println(merged_timers)

#         timer_df = timer_dict_to_df(merged_timers, optimization, rank)

#         all_timer_dfs = vcat(all_timer_dfs, timer_df)
#     end
#     return all_timer_dfs
# end
# original = df_from_h5("test/overall", "original")
# optimized = df_from_h5("test/overall", "optimized")
# union_df = vcat(original, optimized)
# union_df = union_df[union_df.op .== "End-to-End" .|| union_df.op .== "Copy states", :]
# union_df = union_df[union_df.metric .== "time_s", :]
# stats_df = combine(groupby(union_df, [:optimization, :total_rank, :op, :metric]), :value => safe_mean => :mean, :value => safe_std => :std)
# original_stats_df = stats_df[stats_df.optimization .== "original", [:total_rank, :op, :metric, :mean]]
# rename!(original_stats_df, :mean => :original_mean)
# stats_df = leftjoin(stats_df, original_stats_df, on=[:total_rank, :op, :metric])
# stats_df = @chain stats_df begin
#     @transform(
#         improvement = (:original_mean .- :mean) ./ :original_mean * 100
#     )
#     sort!([:total_rank, :optimization, :op])
# end 
# stats_df = stats_df[stats_df.optimization .== "optimized", :]
# println(stats_df)
# function render_stats_refactored(df)
#     # Configuration for plots
#     stats_to_plot = ["End-to-End", "Copy states"]
#     optimizations = unique(df.optimization)
#     n_opts = length(optimizations)

#     # --- Create the individual plots for the grid ---
#     axes = []
#     for (i, stat) in enumerate(stats_to_plot)
#         # Create a new plot object for this subplot
#         p = plot(
#             title = stat * " Time (s)",
#             xlabel = "Total Rank",
#             yaxis = :log, # Use a logarithmic scale for the y-axis
#             legend = false, # Disable individual legends
#             palette = :auto,
#             bottom_margin = 10mm
#         )
        
#         # Add Y-axis label only to the first plot
#         if i == 1
#             plot!(p, ylabel = "Time (log scale)", left_margin = 15mm)
#         end

#         df_filtered = df[(df.op .== stat) .& (df.metric .== "time_s"), :]

#         # Add each optimization as a series to the plot
#         for optimization in optimizations
#             subrk = df_filtered[df_filtered.optimization .== optimization, :]
#             isempty(subrk) && continue

#             g = combine(groupby(subrk, :total_rank),
#                         :value => safe_mean => :mean,
#                         :value => safe_std  => :std)
#             filter!(row -> !ismissing(row.mean) && !ismissing(row.std), g)
#             sort!(g, :total_rank)
#             isempty(g) && continue

#             plot!(p, string.(g.total_rank), g.mean,
#                   ribbon = g.std, seriestype = :path,
#                   markersize = 4, linewidth = 1.5)
#         end
#         push!(axes, p)
#     end

#     # --- Create the global legend ---
#     legend_labels = permutedims(optimizations)
#     legend_plot = plot(
#         (1:n_opts)', # Dummy data to generate legend entries
#         legend = :top,
#         legend_columns = -1, # Forces a single horizontal row
#         labels = legend_labels,
#         framestyle = :none,
#         palette = :auto,
#         size = (1,1) # Minimal size, as it's just for the legend
#     )

#     # --- Combine legend and plots into the final figure ---
#     final_fig = plot(
#         legend_plot,
#         axes...,
#         layout = @layout([A{0.1h}; grid(1, 2)]), # 1x2 grid for the two stats
#         size = (800, 500) # Adjusted size for the 1x2 layout
#     )

#     return final_fig
# end
# # render_stats_refactored(union_df)
# # savefig("real_model.pdf")